In [13]:
# Built-in module.
import logging

# Configure the overall logging system.
# No messages below DEBUG are printed (but all are logged).
logging.basicConfig(
    filename=
    'mylog.log',  # optional: caues logs to go to file (not working for me in notebook though)
    level=logging.DEBUG,
    # Special specifiers understood by logging module.
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Create a logger.
# The name is used to distinguish it in the system.
# Various libraries define their own loggers to use.
logger = logging.getLogger('my_logger')

# Log messages at different levels.
logger.debug('This is a debug message')
logger.info('This is an info message')
logger.warning('This is a warning message')
logger.error('This is an error message')
logger.critical('This is a critical message')

2023-05-21 15:22:48,055 - my_logger - DEBUG - This is a debug message
2023-05-21 15:22:48,057 - my_logger - INFO - This is an info message
2023-05-21 15:22:48,059 - my_logger - WARNING - This is a warning message
2023-05-21 15:22:48,061 - my_logger - ERROR - This is an error message
2023-05-21 15:22:48,062 - my_logger - CRITICAL - This is a critical message
